In [1]:
from llava.mm_utils import get_model_name_from_path, tokenizer_image_token
from llava.model.builder import load_pretrained_model
from llava.constants import IMAGE_TOKEN_INDEX
from llava.eval.run_llava import eval_model
from PIL import Image
from tqdm import tqdm
import numpy as np
import imageio
import torch
import json

[2024-03-05 11:58:50,525] [INFO] [real_accelerator.py:161:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [2]:
file_path = "./youcookii_annotations_test.json"

with open(file_path, 'r') as file:
    data = json.load(file)['database']
long_data = {key: value for key, value in data.items() if len(value['annotations']) >= 5}

In [3]:
model_path = 'liuhaotian/llava-v1.6-34b'

tokenizer, model, image_processor, context_len = load_pretrained_model(
    model_path=model_path,
    model_base=None,
    model_name=get_model_name_from_path(model_path),
    cache_dir='/mnt/esperanto/et/huggingface/hub'
)

You are using a model of type llava to instantiate a model of type llava_llama. This is not supported for all configurations of models and can yield errors.
/home/et/miniconda3/envs/llava_next/lib/python3.11/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
/home/et/miniconda3/envs/llava_next/lib/python3.11/site-packages/torch/nn/modules/module.py:2025: UserWarning: for vision_model.embeddings.class_embedding: copying from a non-meta parameter in the checkpoint to a meta parameter in the current model, which is a no-op. (Did you mean to pass `assign=True` to assign items in the state dictionary to their corresponding key in the module instead of copying them in place?)
  warn

Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

In [4]:
def generate_label(raw_image):
    device = torch.device('cuda')
    prompt = """<|im_start|>user
    <image>
    This image is taken from a recipe. Describe the cooking step in a very short snippet.<|im_end|>
    <|im_start|>assistant"""

    image_tensor = image_processor(raw_image, return_tensors='pt')['pixel_values'].to(device, torch.float16)
    input_ids = tokenizer_image_token(prompt, tokenizer, IMAGE_TOKEN_INDEX, return_tensors='pt').unsqueeze(0).to(device)

    with torch.inference_mode():
        output_ids = model.generate(
                            input_ids,
                            images=image_tensor,
                            do_sample=False,
                            #temperature=0.2,
                            max_new_tokens=30,
                            #use_cache=True,
                            )

    outputs = tokenizer.decode(output_ids[0,:], skip_special_tokens=True).strip().strip('"')
    return outputs

In [5]:
for name, info in tqdm(long_data.items()):
    try:
        video_path = f"raw_videos/{info['subset']}/{name}.mp4"
        video = imageio.get_reader(video_path)
    except:
        continue;

    for i in range(len(long_data[name]['annotations'])):
        index_segment = i
        frame_index = np.random.randint(info['annotations'][index_segment]['segment'][0], info['annotations'][index_segment]['segment'][1]+1)

        fps = video.get_meta_data()['fps']
        frame = Image.fromarray(video.get_data(int(frame_index * fps))).convert('RGB')
        label = generate_label(frame)

        print(label)

        long_data[name]['annotations'][index_segment]['sentence'] = label

    
    while len(long_data[name]['annotations']) < 8:
        index_segment = np.random.randint(0, len(info['annotations']))
        frame_index = np.random.randint(info['annotations'][index_segment]['segment'][0], info['annotations'][index_segment]['segment'][1]+1)

        fps = video.get_meta_data()['fps']
        frame = Image.fromarray(video.get_data(int(frame_index * fps))).convert('RGB')
        label = generate_label(frame)
        
        print(label)

        long_data[name]['annotations'].append({
                                            'segment': [frame_index, frame_index],
                                            'id': len(long_data[name]['annotations']),
                                            'sentence': label,
                                            })
        

  0%|          | 0/188 [00:00<?, ?it/s]

Heat a large pot on the stove.
Adding vegetables to the pot for stir-frying.
Adding liquid to a simmering sauce.
Adding sliced onions to a pot of simmering liquid.
Stirring ingredients in a pot on the stove.
Adding ingredients to a pot on the stove.
Adding tomato sauce to the pan with the chicken


  1%|          | 1/188 [00:13<41:05, 13.18s/it]

Simmering the sauce with onions and spices.
Sprinkle the seasoning evenly over the surface of the food in the pan.
Slice the cake into even pieces.
Adding oil to a hot pan for cooking.
Frying tofu cubes in a pan.
Chopping green onions on a cutting board.
Mixing ingredients in a pot.
Grilling tofu cubes on a griddle.
Sauté tofu cubes in a hot pan with a small amount of oil until golden brown on all sides.
Add stir-fried vegetables to the noodles.
Dipping sauce ready to be served with sushi rolls.


  1%|          | 2/188 [00:31<50:41, 16.35s/it]

...

Frying food on a griddle.
Mixing ingredients in a bowl.
The image shows a person in a kitchen, likely in the process of preparing a dish. They are holding a piece of food, possibly a piece
Adding ingredients to a pot for cooking.
Mixing ingredients in a bowl.
Roll out dough on a floured surface.
Add the chopped onions to the pan and sauté until translucent.
Frying food in a pan with oil.


 99%|█████████▉| 187/188 [39:53<00:10, 10.16s/it]

Mixing ingredients in a bowl.
Peeling and slicing a potato.
Add the marinated meat to the hot pan and sear on all sides.
Frying bacon in a pan.
Chopping green cabbage with a knife.
Chopping onions on a cutting board.
Frying bacon in a pan.
Sauteing chopped onions in a pan.
Sprinkle the seasoning evenly over the surface of the food.
Grate the cheese into the pot.
Rice being stirred in a pot.
Mashing potatoes in a pot.
Combine cooked rice, shredded cabbage, and bacon bits in a bowl.


100%|██████████| 188/188 [40:14<00:00, 12.84s/it]

Add cooked bacon to the mashed potatoes for added flavor and texture.


In [6]:
long_data = {key: value for key, value in long_data.items() if len(value['annotations']) >= 8}

In [8]:
output_path = "./test_data_curated_and_enhanced.json"
with open(output_path, 'w') as file:
    json.dump(long_data, file)